# Base BERT implementation

1. Import dependencies

In [1]:
import random
import time

import numpy as np
import torch
import os

from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

from src.util.torch_device import resolve_torch_device
from src.data.span_detection_ds import ManipulationDetectionDataset
from src.visualization.plot import plot_loss, plot_model_progress, plot_eval_loss
from src.definitions import (
    MODELS_FOLDER,
    RAW_DATA_FOLDER,
    REPORTS_FOLDER,
    PROCESSED_DATA_FOLDER,
)
from src.visualization.ner import visualize_as_markdown_and_save
from src.visualization.reporting import EvaluatingReport
from src.model.span_detection_metrics import compute_metrics

2. Prepare Env

In [2]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

model_checkpoint = "distilbert/distilbert-base-multilingual-cased"

epoch_time = int(time.time())

os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

3. Load dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

dataset_blueprint = ManipulationDetectionDataset(
    tokenizer=tokenizer,
    raw_path=RAW_DATA_FOLDER / "span-detection.parquet",
    processed_path=PROCESSED_DATA_FOLDER / "span-detection",
    seed=random_seed,
)

dataset = dataset_blueprint.read()

3. Prepare model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(dataset_blueprint.label2id),
    id2label=dataset_blueprint.id2label,
    label2id=dataset_blueprint.label2id,
    dropout=0.2,
).to(device)

data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir=MODELS_FOLDER / "manipulation-detector-bert-ner-checkpoint",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    weight_decay=0.3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=random_seed,
    logging_steps=200,
    auto_find_batch_size=True,
    torch_empty_cache_steps=1000,
    metric_for_best_model="f1",
    greater_is_better=True, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics(dataset_blueprint),
)

4. Train model

In [ ]:
trainer.train()

In [ ]:
evaluation_feedback = trainer.evaluate()

In [ ]:
report_file = REPORTS_FOLDER / "span-detection" / "train-report.csv"

report = EvaluatingReport(report_file)

report.write_to_report(evaluation_feedback, epoch_time)

full_report = report.read_report()

full_report.tail(1)

In [ ]:
plot_loss(trainer)

In [ ]:
plot_eval_loss(trainer)

In [ ]:
plot_model_progress(full_report)

5. Save weights

In [ ]:
trainer.save_model(MODELS_FOLDER / "manipulation-detector-bert-ner-model")
tokenizer.save_pretrained(MODELS_FOLDER / "manipulation-detector-bert-ner-model")

6. Test

In [12]:
model = AutoModelForTokenClassification.from_pretrained(
    MODELS_FOLDER / "manipulation-detector-bert-ner-model"
)
tokenizer = AutoTokenizer.from_pretrained(
    MODELS_FOLDER / "manipulation-detector-bert-ner-model"
)

In [ ]:
test_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

test_result = test_pipeline(dataset["test"]["content"])

In [ ]:
visualize_as_markdown_and_save(
    dataset["test"],
    test_result,
    tokenizer,
    REPORTS_FOLDER / "span-detection" / "test-visualization" / f"test-{epoch_time}.md",
)

In [15]:
# [it for it in test_result if len(it) > 0]

In [16]:
# test = dataset["test"][1]

# print(test["content"][0:133])
# print(test["content"][135:250])
# print()
# print(test["content"])
# print(test["labels"])

In [17]:
# dataset["train"][1]

In [18]:
# content = dataset["train"][1]["content"]
# trigger_words = dataset["train"][1]["trigger_words"]

# for it in trigger_words:
#     print(content[it[0]: it[1]])

In [19]:
# labels = dataset["train"][1]['labels']
# input_ids = dataset["train"][1]["input_ids"]

# sub_str = []

# for i in range(len(labels)):
#     if labels[i] == 1:
#         sub_str.append(input_ids[i])

# tokenizer.decode(sub_str)

In [20]:
# tokenizer.convert_ids_to_tokens(dataset["test"][1]["input_ids"])

In [21]:
# tokenizer.decode(dataset["train"][1]["input_ids"])

In [22]:
# from datasets import ClassLabel, Sequence

# dataset["train"].features["labels"] = Sequence(feature=ClassLabel(names=["O", "I-MANIPULATION"]), length=-1, id=None)

# dataset["train"].features["labels"]

In [23]:
# len(dataset["train"]) / 16